<a href="https://colab.research.google.com/github/ksetdekov/trip_choice_optimizer/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from  scipy.stats import gamma, norm

In [2]:
# np.random.seed(0) # фиксируем seed

# risk tolerance
$$\rho = 3.37$$

# initialize

In [3]:
mu_hat_i = 0 # mean
t_i = 0
alpha_i = 0.5
beta_i = 0.5
# rho = 3.37
rho = 1

In [4]:
def update_mean(X, T_last, mu_last):
    mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
    return mu_new

def update_samples(T):
    return T + 1

def update_shape(a):
    return a + 0.5

def update_rate(X, mu_last, beta_last, T_last):
    beta_new = beta_last + (T_last / (T_last + 1)) * (np.square(X - mu_last)) / 2
    return beta_new

In [5]:
hands = pd.DataFrame({'name': ['трамвай + 44', 'пешком + 44', 'мотоцикл', '9 + 44', 'рыжая + жёлтая + салатовая', 
                               'каршеринг', 'серая + салатовая', 'рыжая + фиолетовая + салатовая', 'рыжая + салатовая',
                               'машина']})

In [6]:
hands['mu'] = 0
hands['Te'] = 0
hands['alpha'] = 0.5
hands['beta'] = 0.5

hands

,name,mu,Te,alpha,beta
0,трамвай + 44,0,0,0.5,0.5
1,пешком + 44,0,0,0.5,0.5
2,мотоцикл,0,0,0.5,0.5
3,9 + 44,0,0,0.5,0.5
4,рыжая + жёлтая + салатовая,0,0,0.5,0.5
5,каршеринг,0,0,0.5,0.5
6,серая + салатовая,0,0,0.5,0.5
7,рыжая + фиолетовая + салатовая,0,0,0.5,0.5
8,рыжая + салатовая,0,0,0.5,0.5
9,машина,0,0,0.5,0.5


In [7]:
def to_minutes(timestr: str):
  '''
  convert timestr to float minutes
  '''
  return pd.to_timedelta(timestr).total_seconds()/60

In [8]:
def update_hands(hands, name, value):
    if isinstance(value, str):
        try:
            value = to_minutes(value)
        except ValueError:
            raise ValueError('input time string in hh:mm:ss format')
    elif isinstance(value, float) or isinstance(value, int):
        pass
    else:
        raise ValueError('input time string or int/float value')
    # print(value)
    _, mu, t, alpha, beta = hands[hands.name ==name].values[0]
    beta = update_rate(value, mu, beta, t)
    mu =  update_mean(value, t, mu)
    t = update_samples(t)
    alpha = update_shape(alpha)

    # print(mu, t, alpha, beta)

    hands.loc[hands.name ==name, 'mu'] = mu
    hands.loc[hands.name ==name, 'Te'] = t
    hands.loc[hands.name ==name, 'alpha'] = alpha
    hands.loc[hands.name ==name, 'beta'] = beta

    return hands


In [9]:
hands = update_hands(hands, 'трамвай + 44', 35.03)
hands = update_hands(hands, 'пешком + 44', 40)
hands = update_hands(hands, 'пешком + 44', 36.18)
hands = update_hands(hands, 'пешком + 44', 35.13)
hands = update_hands(hands, 'пешком + 44', 31.65)
hands = update_hands(hands, 'трамвай + 44', 51.23)
hands = update_hands(hands, 'мотоцикл', 20)
hands = update_hands(hands, 'трамвай + 44', 29.65)
hands = update_hands(hands, '9 + 44', 37.63)
hands = update_hands(hands, 'трамвай + 44', 39.58)
hands = update_hands(hands, 'рыжая + жёлтая + салатовая', 47.63)
hands = update_hands(hands, 'каршеринг', 31.75)
hands = update_hands(hands, 'рыжая + салатовая', 52.6)
hands = update_hands(hands, 'каршеринг', 23.15)
hands = update_hands(hands, 'рыжая + фиолетовая + салатовая', 52.82)
hands = update_hands(hands, 'серая + салатовая', 52.37)
hands = update_hands(hands, 'мотоцикл', 21.05)
hands = update_hands(hands, 'мотоцикл', 26.40)
hands = update_hands(hands, 'мотоцикл', "0:25:18")
hands = update_hands(hands, '9 + 44', "0:54:37")
hands = update_hands(hands, 'пешком + 44', "0:38:16")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:51")
hands = update_hands(hands, 'каршеринг', "0:28:04")
hands = update_hands(hands, 'каршеринг', "0:28:05")
hands = update_hands(hands, '9 + 44', "0:31:02")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:58")
hands = update_hands(hands, 'трамвай + 44', "0:30:14")
hands = update_hands(hands, 'мотоцикл', "0:26:29")
hands = update_hands(hands, 'машина', "0:23:59")
hands = update_hands(hands, 'каршеринг', "0:36:41")
hands = update_hands(hands, 'машина', "0:24:49")
hands = update_hands(hands, 'пешком + 44', "0:35:09")


hands

,name,mu,Te,alpha,beta
0,трамвай + 44,37.144667,5,3.0,156.867918
1,пешком + 44,36.062778,6,3.5,21.274216
2,мотоцикл,24.761111,9,5.0,26.324722
3,9 + 44,41.093333,3,2.0,148.539411
4,рыжая + жёлтая + салатовая,47.630000,1,1.0,0.500000
5,каршеринг,29.546667,5,3.0,51.017889
6,серая + салатовая,52.370000,1,1.0,0.500000
7,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000
8,рыжая + салатовая,52.600000,1,1.0,0.500000
9,машина,24.400000,2,1.5,0.673611


: for each t = K + 1, K + 2, . . ., do

7: Sample τi,t from Gamma(αi,t−1, βi,t−1).

8: Sample θi,t from N (ˆµi,t−1, 1/Ti,t−1)

9: Play arm i(t) = arg maxi∈[K] ρθi,t − 1/τi,t and observe reward Xi(t),t

10: Update(ˆµi(t),t−1, Ti(t),t−1, αi(t),t−1, βi(t),t−1)

11: end for


In [10]:
tau = gamma.rvs( a = hands.alpha, scale=1/hands.beta)
theta = norm.rvs(hands.mu, 1/hands.Te)
hands['tau'] = tau
hands['theta'] = theta
hands['SD'] = np.sqrt(1/tau)
hands['var95'] = theta + norm.ppf(1-0.05/2) * hands.SD

hands.reindex(np.argsort(rho * theta + 1/tau))

,name,mu,Te,alpha,beta,tau,theta,SD,var95
9,машина,24.400000,2,1.5,0.673611,3.120381,24.800902,0.566104,25.910445
2,мотоцикл,24.761111,9,5.0,26.324722,0.240166,24.706381,2.040537,28.705761
1,пешком + 44,36.062778,6,3.5,21.274216,0.174949,35.767569,2.390809,40.453468
5,каршеринг,29.546667,5,3.0,51.017889,0.067854,29.467022,3.838962,36.991250
4,рыжая + жёлтая + салатовая,47.630000,1,1.0,0.500000,3.880496,48.178599,0.507641,49.173557
7,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000,0.893776,51.390995,1.057756,53.464159
6,серая + салатовая,52.370000,1,1.0,0.500000,3.996062,53.130651,0.500246,54.111115
8,рыжая + салатовая,52.600000,1,1.0,0.500000,2.118914,54.863045,0.686979,56.209498
0,трамвай + 44,37.144667,5,3.0,156.867918,0.040902,37.324889,4.944557,47.016042
3,9 + 44,41.093333,3,2.0,148.539411,0.008020,40.851716,11.166451,62.737558
